## 앰비언트 에이전트 구축 소개

이 과정의 주요 목표는 앰비언트 에이전트의 개념과 기존 채팅 에이전트와의 차이점을 탐구하는 것입니다.

## LangGraph의 핵심 기본 사항

이 과정은 노드, 엣지, 상태 그래프와 같은 LangGraph의 핵심 기본 사항에 익숙해지도록 설계되었습니다. 이러한 요소들은 우리가 만들 이메일 에이전트의 핵심 구성 요소가 될 것입니다.

다양한 도구 및 플랫폼과의 통합 방법을 시연할 예정입니다. 여기에는 다양한 모델 제공업체와의 통합을 위한 LangChain, 관찰 가능성 및 평가를 위한 LangSmith, 배포를 위한 LangGraph 플랫폼, 시각화 및 디버깅을 위한 LangGraph Studio 사용법이 포함됩니다.

![](./assets/36051.png)

## Resources

- Notebook Reference: langgraph_101.ipynb
- Download Notebook on [GitHub](https://github.com/langchain-ai/agents-from-scratch/blob/main/notebooks/langgraph_101.ipynb)
- For LangSmith Studio: [src/email_assistant](https://github.com/langchain-ai/agents-from-scratch/tree/main/src/email_assistant)
- Slides: [Building Ambient Agents with LangGraph - LangGraph 101.pdf](https://files.cdn.thinkific.com/file_uploads/967498/attachments/b9a/35b/890/Building_Ambient_Agents_with_LangGraph_-_LangGraph_101.pdf)

## 설치하기

In [1]:
%pip install -Uq langchain

Note: you may need to restart the kernel to use updated packages.


## 환경 설정

In [2]:
from dotenv import load_dotenv


load_dotenv("../../.env", override=True)

False

In [3]:
import os


os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_PROJECT"] = "langchain-academy"

## Chat models

In [6]:
from langchain.chat_models import init_chat_model


llm = init_chat_model("openai:gpt-4.1-mini", temperature=0)

## 도구(Tools)

도구는 채팅 모델이 호출할 수 있는 유틸리티입니다. LangChain에서는 `@tool` 데코레이터를 사용하여 도구를 생성할 수 있으며, 이는 Python 함수를 호출 가능한 도구로 변환합니다. 이 데코레이터는 함수 정의로부터 도구의 이름, 설명 및 예상 인수를 자동으로 추론합니다. 또한 모델 컨텍스트 프로토콜(MCP) 서버를 LangChain 호환 도구로 사용할 수도 있습니다.

In [8]:
from langchain.tools import tool


@tool
def write_email(to: str, subject: str, content: str):
    """이메일을 작성하고 전송합니다."""
    return f"Email sent to {to} with subject '{subject}' and content: {content}"

In [ ]:
type(write_email)

langchain_core.tools.structured.StructuredTool

In [10]:
write_email.args

{'to': {'title': 'To', 'type': 'string'},
 'subject': {'title': 'Subject', 'type': 'string'},
 'content': {'title': 'Content', 'type': 'string'}}

In [11]:
write_email.description

'이메일을 작성하고 전송합니다.'

## Tool Calling

도구는 대규모 언어 모델(LLM)에 의해 호출될 수 있습니다. 도구가 모델에 바인딩되면, 모델은 도구 인자를 포함한 구조화된 출력을 반환함으로써 도구를 호출할 수 있습니다. `bind_tools` 메서드를 사용하여 대규모 언어 모델에 도구를 추가합니다.

프로바이더는 특정 도구를 호출하도록 강제하기 위해 [`tool_choice`와 같은 매개변수](https://python.langchain.com/docs/how_to/tool_choice/)를 사용할 수 있습니다. `any`는 도구 중 최소 하나를 선택합니다.

또한 [`parallel_tool_calls=False`를 설정](https://python.langchain.com/docs/how_to/tool_calling_parallel/)하여 모델이 한 번에 하나의 도구만 호출하도록 보장할 수 있습니다.

In [15]:
model_with_tools = llm.bind_tools(
    [write_email],
    tool_choice="any",
    parallel_tool_calls=False,
)

In [16]:
output = model_with_tools.invoke(
    "내일 회의에 대해 상사(boss@company.ai)에게 보낼 답변 초안을 작성해 주세요."
)

In [17]:
type(output)

langchain_core.messages.ai.AIMessage

In [18]:
output

AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 105, 'prompt_tokens': 78, 'total_tokens': 183, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4.1-mini-2025-04-14', 'system_fingerprint': 'fp_4c2851f862', 'id': 'chatcmpl-CT82YOxTOyNnmdrggFMfFSabXZvys', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--0fd07ac8-d814-4cce-b187-ac2e3ab0dd52-0', tool_calls=[{'name': 'write_email', 'args': {'to': 'boss@company.ai', 'subject': '내일 회의 관련 답변', 'content': '안녕하세요, 상사님.\n\n내일 회의에 대해 확인하였습니다. 준비된 자료와 안건을 바탕으로 적극적으로 참여하겠습니다. 혹시 추가로 요청하실 사항이나 논의하고 싶은 주제가 있으시면 미리 알려주시면 감사하겠습니다.\n\n감사합니다.\n\n[당신의 이름]'}, 'id': 'call_vQQAtQsOgOnalfvzOtzumBCg', 'type': 'tool_call'}], usage_metadata={'input_tokens': 7

In [20]:
args = output.tool_calls[0]['args']
args

{'to': 'boss@company.ai',
 'subject': '내일 회의 관련 답변',
 'content': '안녕하세요, 상사님.\n\n내일 회의에 대해 확인하였습니다. 준비된 자료와 안건을 바탕으로 적극적으로 참여하겠습니다. 혹시 추가로 요청하실 사항이나 논의하고 싶은 주제가 있으시면 미리 알려주시면 감사하겠습니다.\n\n감사합니다.\n\n[당신의 이름]'}

In [30]:
from IPython.display import Markdown


result = write_email.invoke(args)
Markdown(result)

Email sent to boss@company.ai with subject '내일 회의 관련 답변' and content: 안녕하세요, 상사님.

내일 회의에 대해 확인하였습니다. 준비된 자료와 안건을 바탕으로 적극적으로 참여하겠습니다. 혹시 추가로 요청하실 사항이나 논의하고 싶은 주제가 있으시면 미리 알려주시면 감사하겠습니다.

감사합니다.

[당신의 이름]

## Workflows

LLM을 활용한 애플리케이션 구축에는 다양한 패턴이 존재합니다.

[사전 정의된 워크플로우에 LLM 호출함으로써](https://langchain-ai.github.io/langgraph/tutorials/workflows/) 시스템이 더 많은 의사 결정 권한을 가질 수 있습니다.

이메일을 작성할지 여부를 결정하기 위한 라우터 단계를 추가할 수 있습니다.


에이전트는 사전에 필요한 정확한 단계를 예측하기 어려운 개방형 문제에 적합합니다.

워크플로는 제어 흐름을 사전에 쉽게 정의할 수 있을 때 종종 적합합니다.

## Agents

